In [7]:
import sys
from os import getenv
from pathlib import Path

# Set the base path
base_path = Path("../")  # One level up from the current working directory

# Add the src/ directory to sys.path using base_path
sys.path.append(str((base_path / "src").resolve()))

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from utils.yaml_functions import load_yaml

In [16]:
problem = load_yaml(base_path / "decision_problems/oil_field_purchase/problem.yaml")["long_version"]

objective = """
## Objective

You are acting as a decision analysis expert. Based on the problem description above, your task is to:

1. Analyze the decision problem using principles of decision theory.
2. Compute the expected utility of all relevant options, considering both the possibility of testing and not testing.
3. Determine whether the company should perform the geological test.
4. Based on the test result (if the test is chosen), decide whether the company should purchase the oil field.
5. Recommend the optimal strategy that maximizes expected value for the company.

Make sure your reasoning is explicit, and show all calculations, including conditional probabilities, expected utilities, and decision comparisons. Your final answer should include:
- The recommended overall strategy (test or not, and what to do based on the outcome)
- Justification with relevant numeric reasoning
"""

prompt_str = "{problem}" + "\n" + objective

In [18]:
print(prompt_str)

## Oil Field Investment Decision Problem  

### Problem Overview  
An oil company must decide whether to **buy (B)** an oil field whose true quality **Q** is uncertain.  

Q has three possible states:
- **High**: Exceptional reserves, high-profit potential  
- **Medium**: Moderate reserves, acceptable profit  
- **Low**: Poor reserves, high financial risk  

Historical and survey data give the prior probabilities:
- P(Q = High) = 0.30  
- P(Q = Medium) = 0.40  
- P(Q = Low)   = 0.30  

### Information-Gathering Option  
The company may optionally perform a **Porosity Test** at cost \$0.01 B (i.e.\ \$10 million):  
- **Result R = “Pass”** if porosity ≥ 15%  
- **Result R = “Fail”** if porosity < 15%  
- Or skip testing and go straight to the purchase decision.

#### Test Reliability 

The test provides imperfect but valuable information about the true quality of the field. We can represent the reliability of the test as a conditional probability distribution P(R | Q) where:

**High Qual

In [17]:
model_openrouter = "gpt-4o-mini"
request_timeout = 10

# Create a prompt template
prompt = PromptTemplate(template=prompt_str, input_variables=["text"])

# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model_name=model_openrouter,
    request_timeout=request_timeout,
)

# Initialize the output parser
output_parser = StrOutputParser()

# Create a chain for processing the text
chain = prompt | llm | output_parser

In [36]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from os import getenv

model_openrouter = "gpt-4o-mini"
request_timeout = 10

prompt_str = "Say hello and tell me your name."

# Create a prompt template
prompt = PromptTemplate(template=prompt_str, input_variables=[])

# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model_name=model_openrouter,
    request_timeout=request_timeout,
)

# Initialize the output parser
output_parser = StrOutputParser()

# Create a chain for processing the text
chain = prompt | llm | output_parser

# Run the chain and print the output
response = chain.invoke({})
print(response)

Hello! I’m ChatGPT, your AI assistant. How can I help you today?


In [42]:
import math
import numexpr
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
import os

@tool
def calculator(expression: str) -> str:
    """Calculate expression using Python's numexpr library.

    Expression should be a single line mathematical expression
    that solves the problem.

    Examples:
        "37593 * 67" for "37593 times 67"
        "37593**(1/5)" for "37593^(1/5)"
    """
    local_dict = {"pi": math.pi, "e": math.e}
    return str(
        numexpr.evaluate(
            expression.strip(),
            global_dict={},  # restrict access to globals
            local_dict=local_dict,  # add common mathematical functions
        )
    )

tools = [calculator]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # <-- Use OPENAI_TOOLS instead of OPENAI_FUNCTIONS
    verbose=True,
)

query = """
Given P(A) = 0.3, P(B|A) = 0.8, and P(B) = 0.5, calculate P(A|B) using Bayes theorem.
Then calculate P(not A | B).
"""

print(agent.run(query))




> Entering new AgentExecutor chain...


ValueError: {'message': '"functions" and "function_call" are deprecated in favor of "tools" and "tool_choice." To learn how to use tools, visit: https://openrouter.ai/docs/requests#tool-calls', 'code': 400, 'metadata': {'provider_name': 'Azure'}}

In [61]:
import json, requests
from openai import OpenAI

# You can use any model that supports tool calling
# MODEL = "google/gemini-2.0-flash-001"
MODEL = "gpt-4o-mini"

openai_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("OPENROUTER_API_KEY"),
)

task = "What are the titles of some James Joyce books?"

messages = [
  {
    "role": "system",
    "content": "You are a helpful assistant."
  },
  {
    "role": "user",
    "content": task,
  }
]

def search_gutenberg_books(search_terms):
    search_query = " ".join(search_terms)
    url = "https://gutendex.com/books"
    response = requests.get(url, params={"search": search_query})

    simplified_results = []
    for book in response.json().get("results", []):
        simplified_results.append({
            "id": book.get("id"),
            "title": book.get("title"),
            "authors": book.get("authors")
        })

    return simplified_results

tools = [
  {
    "type": "function",
    "function": {
      "name": "search_gutenberg_books",
      "description": "Search for books in the Project Gutenberg library based on specified search terms",
      "parameters": {
        "type": "object",
        "properties": {
          "search_terms": {
            "type": "array",
            "items": {
              "type": "string"
            },
            "description": "List of search terms to find books in the Gutenberg library (e.g. ['dickens', 'great'] to search for books by Dickens with 'great' in the title)"
          }
        },
        "required": ["search_terms"]
      }
    }
  }
]

TOOL_MAPPING = {
    "search_gutenberg_books": search_gutenberg_books
}

request_1 = {
    "model": "google/gemini-2.0-flash-001",
    "tools": tools,
    "messages": messages
}


In [62]:
response_1 = openai_client.chat.completions.create(**request_1)
message = response_1.choices[0].message

print(message)

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tool_0_search_gutenberg_books', function=Function(arguments='{"search_terms":["james","joyce"]}', name='search_gutenberg_books'), type='function', index=0)], reasoning=None)


In [63]:
message.tool_calls

[ChatCompletionMessageToolCall(id='tool_0_search_gutenberg_books', function=Function(arguments='{"search_terms":["james","joyce"]}', name='search_gutenberg_books'), type='function', index=0)]

In [64]:
# Append the response to the messages array so the LLM has the full context
# It's easy to forget this step!
messages.append(message)

# Now we process the requested tool calls, and use our book lookup tool
for tool_call in message.tool_calls:
    '''
    In this case we only provided one tool, so we know what function to call.
    When providing multiple tools, you can inspect `tool_call.function.name`
    to figure out what function you need to call locally.
    '''
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)
    tool_response = TOOL_MAPPING[tool_name](**tool_args)
    messages.append({
      "role": "tool",
      "tool_call_id": tool_call.id,
      "name": tool_name,
      "content": json.dumps(tool_response),
    })

In [65]:
request_2 = {
  "model": MODEL,
  "messages": messages,
  "tools": tools
}

response_2 = openai_client.chat.completions.create(**request_2)

print(response_2.choices[0].message.content)


Here are some titles of books by James Joyce:

1. **Ulysses**
2. **Dubliners**
3. **A Portrait of the Artist as a Young Man**
4. **Chamber Music**
5. **Exiles: A Play in Three Acts**
6. **Index of the Project Gutenberg Works of James Joyce**

These are notable works by Joyce, reflecting his contributions to literature.


In [77]:
import json

def run_tool_call_loop(openai_client, model, tools, messages, tool_mapping, max_iterations=5):
    """
    Repeatedly calls the LLM, handles any tool requests, and returns final response.

    :param openai_client: Initialized OpenAI client for OpenRouter.
    :param model: The model to use (e.g., 'openai/gpt-4', 'google/gemini-2.0-pro').
    :param tools: List of available tool specifications (OpenAI-style function definitions).
    :param messages: Initial message list (system + user prompts).
    :param tool_mapping: Dictionary mapping tool names to Python functions.
    :param max_iterations: Safety cap to avoid infinite loops.
    :return: Final message content from the LLM.
    """
    for iteration in range(max_iterations):
        print(f"\n--- Iteration {iteration + 1} ---")

        response = openai_client.chat.completions.create(
            model=model,
            tools=tools,
            messages=messages
        )
        
        msg = response.choices[0].message
        messages.append(msg.model_dump())

        # If no tool calls, print and exit
        if not getattr(msg, "tool_calls", None):
            print("🔚 Final model output:\n", msg.content[:500])
            return msg.content, messages

        # Tool call(s) detected
        for tool_call in msg.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            tool_func = tool_mapping.get(tool_name)

            if not tool_func:
                raise ValueError(f"No tool function mapped for '{tool_name}'")

            print(f"🛠️  Model requested tool: {tool_name} with args: {tool_args}")
            tool_result = tool_func(**tool_args)

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": tool_name,
                "content": json.dumps(tool_result),
            })

    print("⚠️ Max iterations reached without final response.")
    return None


In [78]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=getenv("OPENROUTER_API_KEY")
)

# Messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What are the titles of some Charles Dickens books?"}
]

# Tools + Mapping
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_gutenberg_books",
            "description": "Search for books in the Project Gutenberg library.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_terms": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Search terms for book lookup"
                    }
                },
                "required": ["search_terms"]
            }
        }
    }
]

def search_gutenberg_books(search_terms):
    import requests
    resp = requests.get("https://gutendex.com/books", params={"search": " ".join(search_terms)})
    return [{"title": b["title"]} for b in resp.json().get("results", [])[:5]]

TOOL_MAPPING = {
    "search_gutenberg_books": search_gutenberg_books
}


In [76]:
# Run it
result = run_tool_call_loop(
    openai_client=client,
    model="gpt-4o-mini",  # or any tool-calling-capable model
    tools=tools,
    messages=messages,
    tool_mapping=TOOL_MAPPING
)


--- Iteration 1 ---


/var/folders/47/gybx85kd6f147h3rp8wgdz9w0000gn/T/ipykernel_9660/1360868788.py:25: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  messages.append(msg.dict())


🛠️  Model requested tool: search_gutenberg_books with args: {'search_terms': ['Charles Dickens']}

--- Iteration 2 ---
🔚 Final model output:
 Here are some titles of books by Charles Dickens:

1. A Tale of Two Cities
2. Great Expectations
3. A Christmas Carol in Prose; Being a Ghost Story of Christmas
4. Oliver Twist
5. Hard Times


In [82]:
len(result[1])

5

In [91]:
result[1][4]

{'content': 'Here are some titles of books by Charles Dickens:\n\n1. A Tale of Two Cities\n2. Great Expectations\n3. A Christmas Carol in Prose; Being a Ghost Story of Christmas\n4. Oliver Twist\n5. Hard Times',
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None,
 'reasoning': None}